In [1]:
# ============================================
# Phase 5: Generate Predictions
# ============================================

import pandas as pd
import joblib
from pathlib import Path

# Set project root
BASE = Path(r"C:\Users\lavan\OneDrive\Desktop\PredictiveMaintenanceProject")
ST   = BASE / "data" / "staging"

# Load model + features
model = joblib.load(BASE / "models" / "rf_model.pkl")
feat = pd.read_csv(ST / "features_for_model.csv", parse_dates=["Date"])

# Predictions
X = feat[["HoursUsed", "Hours_7d_mean", "Temperature", "Vibration", "DaysSinceLastMaint"]].fillna(0)
feat["pred_prob"] = model.predict_proba(X)[:, 1]
feat["priority"] = feat["pred_prob"] * (1 + feat["DaysSinceLastMaint"] / 365)

# Keep latest row per device
latest = feat.sort_values(["DeviceID", "Date"]).groupby("DeviceID").tail(1)
out = latest[["DeviceID", "Date", "pred_prob", "priority"]]

# Save predictions
out.to_csv(ST / "predictions_latest.csv", index=False)
print("✅ Predictions saved to:", ST / "predictions_latest.csv")


✅ Predictions saved to: C:\Users\lavan\OneDrive\Desktop\PredictiveMaintenanceProject\data\staging\predictions_latest.csv
